Подготовим данные для комфортного построения моделей и комфортного EDA в будущем. Для этого создадим несколько дополнительных колонок, удалим неинформативные.

# Импортируем все необходимые библиотеки

In [3]:
import sys
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

# Переменные и словарь ICD-10 для удобства

In [29]:
# VARS

PATH = '.\\data\\' if sys.platform == 'win32' else './data/'

# Введем переменную, которая поможет лучше понимать, входит ли временной интервал в до или после ковидный период
COVID_BEGAN = '2019-11-30'

Словарь соответствует последней редакции с сайта mkb-10.com

In [5]:
ICD_10_chapter_description = {
    'I': 'Некоторые инфекционные и паразитарные болезни',
    'II':  'Новообразования',
    'III':  'Болезни крови, кроветворных органов и отдельные нарушения, вовлекающие иммунный механизм',
    'IV':  'Болезни эндокринной системы, расстройства питания и нарушения обмена веществ',
    'V':  'Психические расстройства и расстройства поведения',
    'VI':  'Болезни нервной системы',
    'VII':  'Болезни глаза и его придаточного аппарата',
    'VIII':  'Болезни уха и сосцевидного отростка',
    'IX':  'Болезни системы кровообращения',
    'X':  'Болезни органов дыхания',
    'XI':  'Болезни органов пищеварения',
    'XII':  'Болезни кожи и подкожной клетчатки',
    'XIII':  'Болезни костно-мышечной системы и соединительной ткани',
    'XIV':  'Болезни мочеполовой системы',
    'XV':  'Беременность, роды и послеродовой период',
    'XVI':  'Отдельные состояния, возникающие в перинатальном периоде',
    'XVII':  'Врожденные аномалии [пороки развития], деформации и хромосомные нарушения',
    'XVIII':  'Симптомы, признаки и отклонения от нормы, выявленные при клинических и лабораторных исследованиях, не классифицированные в других рубриках',
    'XIX':  'Травмы, отравления и некоторые другие последствия воздействия внешних причин',
    'XX':  'Внешние причины заболеваемости и смертности',
    'XXI':  'Факторы, влияющие на состояние здоровья населения и обращения в учреждения здравоохранения',
    'XXII':  'Коды для особых целей'
}

ICD_10_chapter = {
    'I': ['A00', 'B99'],
    'II': ['C00', 'D48'],
    'III': ['D50','D89'],
    'IV': ['E00','E90'],
    'V': ['F00','F99'],
    'VI': ['G00','G99'],
    'VII': ['H00','H59'],
    'VIII': ['H60','H95'],
    'IX': ['I00','I99'],
    'X': ['J00','J99'],
    'XI': ['K00','K93'],
    'XII': ['L00','L99'],
    'XIII': ['M00','M99'],
    'XIV': ['N00','N99'],
    'XV': ['O00','O99'],
    'XVI': ['P00','P96'],
    'XVII': ['Q00','Q99'],
    'XVIII': ['R00','R99'],
    'XIX': ['S00','T98'],
    'XX': ['V01','Y98'],
    'XXI': ['Z00','Z99'],
    'XXII': ['U00','U85']
}

Введем функции для получения описания главы и получения главы по коду МКБ

In [6]:
def get_chapter(code):
    for cls, limits in ICD_10_chapter.items():
        if limits[0] <= code[:3] <= limits[1]:
            return cls
    return None

def get_description(chapter):
    return ICD_10_chapter_description[chapter]

# Подготовим данные

In [7]:
df = pd.read_csv(''.join([PATH, 'train_dataset_train.csv']), sep=';')
# Make a copy
orig_df = df.copy()

In [8]:
df.head(10)

,PATIENT_SEX,MKB_CODE,ADRES,VISIT_MONTH_YEAR,AGE_CATEGORY,PATIENT_ID_COUNT
0,0,A00.0,Гурьевск,8.21,young,1
1,0,A00.0,Калининград,3.20,children,1
2,0,A00,Гусев,3.19,children,1
3,0,A00,Калининград,1.22,children,1
4,0,A00,Калининград,2.18,children,1
5,0,A00,Калининград,3.22,children,4
6,0,A00,Калининград,3.22,elderly,1
7,0,A00,Калининград,3.22,middleage,1
8,0,A00,Калининград,3.22,young,3
9,0,A00,Калининград,7.18,young,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212393 entries, 0 to 2212392
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   PATIENT_SEX       int64  
 1   MKB_CODE          object 
 2   ADRES             object 
 3   VISIT_MONTH_YEAR  float64
 4   AGE_CATEGORY      object 
 5   PATIENT_ID_COUNT  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 101.3+ MB


In [10]:
df.shape

(2212393, 6)

Повысим информативность датасета путем замена бинарной переменной на категориальную. В виду того, что планируется использовать CatBoost для построения модели - это не должно стать проблемой

In [11]:
df.PATIENT_SEX = df.PATIENT_SEX.map({1:'Male', 0:'Female'})

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212393 entries, 0 to 2212392
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   PATIENT_SEX       object 
 1   MKB_CODE          object 
 2   ADRES             object 
 3   VISIT_MONTH_YEAR  float64
 4   AGE_CATEGORY      object 
 5   PATIENT_ID_COUNT  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 101.3+ MB


In [13]:
# Создадим колонки с датой, для более удобного анализа. 
# Колонка со днём неинформативна и нужна только для корректной инициализации столбца DATE объектом datetime 

df['DAY'] = [1] * df.shape[0]
df['MONTH'] = [value[0] for value in df.VISIT_MONTH_YEAR.astype(str).apply(lambda x: x.split('.'))]
df['MONTH'] = df['MONTH'].astype('int64')
df['YEAR'] = ['20' + value[1] if len(value[1]) > 1 else '20' + value[1] + '0' for value in df.VISIT_MONTH_YEAR.astype(str).apply(lambda x: x.split('.'))]
df['YEAR'] = df['YEAR'].astype('int64')
df['DATE'] = pd.to_datetime(df[['DAY', 'MONTH', 'YEAR']])
df = df.drop('VISIT_MONTH_YEAR', axis=1)
df = df.drop('DAY', axis=1)

In [14]:
df.head(10)

,PATIENT_SEX,MKB_CODE,ADRES,AGE_CATEGORY,PATIENT_ID_COUNT,MONTH,YEAR,DATE
0,Female,A00.0,Гурьевск,young,1,8,2021,2021-08-01
1,Female,A00.0,Калининград,children,1,3,2020,2020-03-01
2,Female,A00,Гусев,children,1,3,2019,2019-03-01
3,Female,A00,Калининград,children,1,1,2022,2022-01-01
4,Female,A00,Калининград,children,1,2,2018,2018-02-01
5,Female,A00,Калининград,children,4,3,2022,2022-03-01
6,Female,A00,Калининград,elderly,1,3,2022,2022-03-01
7,Female,A00,Калининград,middleage,1,3,2022,2022-03-01
8,Female,A00,Калининград,young,3,3,2022,2022-03-01
9,Female,A00,Калининград,young,1,7,2018,2018-07-01


Соотнесем главу МКБ с каждым наблюдением

In [15]:
df['CHAPTER'] = [get_chapter(code) for code in df.MKB_CODE]
df.head(10)

,PATIENT_SEX,MKB_CODE,ADRES,AGE_CATEGORY,PATIENT_ID_COUNT,MONTH,YEAR,DATE,CHAPTER
0,Female,A00.0,Гурьевск,young,1,8,2021,2021-08-01,I
1,Female,A00.0,Калининград,children,1,3,2020,2020-03-01,I
2,Female,A00,Гусев,children,1,3,2019,2019-03-01,I
3,Female,A00,Калининград,children,1,1,2022,2022-01-01,I
4,Female,A00,Калининград,children,1,2,2018,2018-02-01,I
5,Female,A00,Калининград,children,4,3,2022,2022-03-01,I
6,Female,A00,Калининград,elderly,1,3,2022,2022-03-01,I
7,Female,A00,Калининград,middleage,1,3,2022,2022-03-01,I
8,Female,A00,Калининград,young,3,3,2022,2022-03-01,I
9,Female,A00,Калининград,young,1,7,2018,2018-07-01,I


Переведем переменную AGE_CATEGORY в конкретные интервалы

In [16]:
df['AGE_CATEGORY'] = df['AGE_CATEGORY'].map({
    'children': '0-18',
    'young': '18-44',
    'middleage': '45-59',
    'elderly': '60-74', 
    'old': '75-90',
    'centenarians': '90+'
}) 

Переменная IS_COVID подскажет приходится ли наблюдение на период пандемии COVID-19

In [17]:
df['IS_COVID'] = [True if date >= pd.Timestamp(COVID_BEGAN) else False for date in df.DATE]

In [18]:
df.head()

,PATIENT_SEX,MKB_CODE,ADRES,AGE_CATEGORY,PATIENT_ID_COUNT,MONTH,YEAR,DATE,CHAPTER,IS_COVID
0,Female,A00.0,Гурьевск,18-44,1,8,2021,2021-08-01,I,True
1,Female,A00.0,Калининград,0-18,1,3,2020,2020-03-01,I,True
2,Female,A00,Гусев,0-18,1,3,2019,2019-03-01,I,False
3,Female,A00,Калининград,0-18,1,1,2022,2022-01-01,I,True
4,Female,A00,Калининград,0-18,1,2,2018,2018-02-01,I,False


Есть гипотеза, что для минимизации ошибки важно не столько правильно и точно предсказывать значение таргета, сколько, прежде всего, правильно и точно предсказывать промежуток (интервал), в которое будет попадать значение таргета. Для этого введем интервалы (пока что приблизительно и на глаз), и соотнесем значения целевой переменной каждого наблюдения с этими интервалами. 

In [19]:
match_dict = {
    '1-10': [i for i in range(11)],
    '10-100': [i for i in range(10, 101)],
    '100-1000': [i for i in range(100, 1001)],
    '1000-10000': [i for i in range(1000, 10001)],
    '10000+': [i for i in range(10000, 13533)]
}

def get_key(item):
    for key, value in match_dict.items():
        if item in value:
            return key

In [20]:
df['TARGET_RANGE'] = [get_key(id) for id in df.PATIENT_ID_COUNT]

Переопределим порядок колонок. для более удобного визуального восприятия

In [21]:
df = df = df.reindex(columns=[
    'PATIENT_SEX', 'ADRES', 'MKB_CODE', 'CHAPTER', 'AGE_CATEGORY', 
    'MONTH', 'YEAR', 'DATE', 'IS_COVID', 'PATIENT_ID_COUNT','TARGET_RANGE'])


In [22]:
df.head(10)

,PATIENT_SEX,ADRES,MKB_CODE,CHAPTER,AGE_CATEGORY,MONTH,YEAR,DATE,IS_COVID,PATIENT_ID_COUNT,TARGET_RANGE
0,Female,Гурьевск,A00.0,I,18-44,8,2021,2021-08-01,True,1,1-10
1,Female,Калининград,A00.0,I,0-18,3,2020,2020-03-01,True,1,1-10
2,Female,Гусев,A00,I,0-18,3,2019,2019-03-01,False,1,1-10
3,Female,Калининград,A00,I,0-18,1,2022,2022-01-01,True,1,1-10
4,Female,Калининград,A00,I,0-18,2,2018,2018-02-01,False,1,1-10
5,Female,Калининград,A00,I,0-18,3,2022,2022-03-01,True,4,1-10
6,Female,Калининград,A00,I,60-74,3,2022,2022-03-01,True,1,1-10
7,Female,Калининград,A00,I,45-59,3,2022,2022-03-01,True,1,1-10
8,Female,Калининград,A00,I,18-44,3,2022,2022-03-01,True,3,1-10
9,Female,Калининград,A00,I,18-44,7,2018,2018-07-01,False,1,1-10


Переменную ADRES оставим без изменений. Согласно ответам из чата от организаторов все упомянутые города - города Калининградской области. 

In [23]:
df.to_csv(PATH+'prepared_df.csv')